In [2]:
import sqlite3
import pandas as pd

In [3]:
connect = sqlite3.connect("dog.db")

In [4]:
cursor = connect.cursor()

In [191]:
cursor.execute("DROP TABLE IF EXISTS parents")

In [192]:
cursor.execute("CREATE TABLE parents (parent VARCHAR(20), child VARCHAR(20));")

In [193]:
cursor.execute('''
INSERT INTO parents
(parent, child)
VALUES ("abraham", "barack") UNION
VALUES ("abraham", "clinton") UNION
VALUES ("delano", "herbert") UNION
VALUES  ("fillmore", "abraham") UNION
VALUES ("fillmore", "delano") UNION
VALUES ("fillmore",   "grover") UNION
VALUES ("eisenhower", "fillmore");''')

In [194]:
pd.read_sql_query('SELECT * FROM parents', connect)

,parent,child
0,abraham,barack
1,abraham,clinton
2,delano,herbert
3,eisenhower,fillmore
4,fillmore,abraham
5,fillmore,delano
6,fillmore,grover


In [195]:
pd.read_sql_query('SELECT * FROM parents where parent="abraham" ', connect)

,parent,child
0,abraham,barack
1,abraham,clinton


In [196]:
pd.read_sql_query("SELECT child FROM parents where child LIKE '%e%'", connect)

,child
0,herbert
1,fillmore
2,delano
3,grover


In [197]:
d = pd.read_sql_query('SELECT DISTINCT parent FROM parents ORDER BY parent desc', connect)
d

,parent
0,fillmore
1,eisenhower
2,delano
3,abraham


In [198]:
for i in range(len(d)-1,-1,-1):
    e = pd.read_sql_query('SELECT child FROM parents where parent="'+str(d.parent[i])+'"', connect)
    if(len(e)>1):
        print(e,"\n")

     child
0   barack
1  clinton 

     child
0  abraham
1   delano
2   grover 



In [199]:
cursor.execute("DROP TABLE IF EXISTS dogs")

In [200]:
query = '''CREATE TABLE dogs AS
SELECT "abraham" AS name,"long" AS fur UNION
SELECT "barack","short" UNION
SELECT "clinton", "long" UNION
SELECT "delano", "long" UNION
SELECT "eisenhower", "short" UNION
SELECT "fillmore", "curly" UNION
SELECT "grover", "short" UNION
SELECT "herbert", "curly";'''
cursor.execute(query)

In [214]:
pd.read_sql_query('SELECT COUNT(*) FROM dogs where fur="short"', connect)

,COUNT(*)
0,3


In [219]:
pd.read_sql_query('SELECT parent FROM parents JOIN dogs WHERE parents.child=dogs.name AND dogs.fur="curly"',connect)

,parent
0,eisenhower
1,delano


In [232]:
par = pd.read_sql_query('SELECT parent,child,fur FROM parents JOIN dogs where parents.parent=dogs.name',connect)
chi = pd.read_sql_query('SELECT child,fur FROM parents JOIN dogs where parents.child=dogs.name',connect)
for i in range(0,len(par)):
    if(par.child[i]==chi.child[i]):
        if(par.fur[i]==chi.fur[i]):
            print(" child --> parent\n",chi.child[i],"-->",par.parent[i])

 child --> parent
 clinton --> abraham


In [204]:
cursor.execute("DROP TABLE IF EXISTS animals")

In [205]:
query = '''create table animals as  select "dog" as kind, 4 as legs, 20 as weight union  select "cat" , 4 , 10 union  select "ferret" , 4 , 10 union  select "parrot" , 2 , 6 union  select "penguin" , 2 , 10 union select "t-rex" , 2 , 12000;'''
cursor.execute(query)

In [213]:
pd.read_sql_query('SELECT kind, MIN(weight) FROM animals;', connect)

,kind,MIN(weight)
0,parrot,6


In [207]:
pd.read_sql_query('SELECT AVG(legs),AVG(weight) FROM animals;',connect)

,AVG(legs),AVG(weight)
0,3.0,2009.333333


In [208]:
pd.read_sql_query('SELECT * FROM animals where legs>2 AND weight<20;', connect)

,kind,legs,weight
0,cat,4,10
1,ferret,4,10


In [209]:
pd.read_sql_query('SELECT AVG(weight) FROM animals GROUP BY legs;',connect)

,AVG(weight)
0,4005.333333
1,13.333333
